<a href="https://colab.research.google.com/github/cloud-commander/face-mask-detection/blob/master/2_Prepare_Data_Part2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


<a href="https://blog.cloudcommander.net" target="_parent"><img src="https://raw.githubusercontent.com/cloud-commander/hexoblog/master/cloud.png" alt="Visit my Blog">
</a>
<br> 
# <span style="font-family:Didot; font-size:3em;"> Cloud Commander </span>


<a href="https://colab.research.google.com/github/cloud-commander/face-mask-detection/blob/master/2_Prepare_Data_Part2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab"></a>
&nbsp;&nbsp;&nbsp;&nbsp;
<a href="https://github.com/cloud-commander/face-mask-detection/blob/master/2_Prepare_Data_Part2.ipynb" target="_parent"><img src="https://img.shields.io/static/v1?logo=GitHub&label=&color=333333&style=flat&message=View%20on%20GitHub" alt="View in GitHub"></a>


# Automatic Face Image Annotation & Masked Photo Generation ##

In the previous notebook, we prepared the unmasked photos in the dataset. In this notebook we will look at pre-processing the images for our masked photo dataset.

Firstly we will identify the bounding boxes for the faces as with the unmasked photos but additionally we will then also place a series of masked into those faces thus making them masked photos.

## Import required libraries

In [ ]:
!wget https://raw.githubusercontent.com/cloud-commander/face-mask-detection/master/config/constants.py
from constants import * 
!wget {ANNOTATE}
!wget {PLACE_MASKS}
!wget {MOVE_FILES}

!pip install face_recognition
!pip install wget

from annotate import *
from place_masks import * 
from move_files import * 

import os
import glob
import tarfile

import sys
import random

## Prepare dataset ###

### Connect to Google Drive


In [ ]:
from google.colab import drive

drive.mount('/content/drive/')

### Download faces dataset

In [ ]:
%cd {DATASET_DIR_TMP1}
!wget https://raw.githubusercontent.com/cloud-commander/face-mask-detection/master/data/100_faces_320x320_dataset_set_2.zip
!unzip -j 100_faces_320x320_dataset_set_2.zip '*.jpg'
!rm 100_faces_320x320_dataset_set_2.zip

## Annotate the images
Calls the annotate function to automatically annotate the faces in our image folder and outputs XML files

In [ ]:
annotate(DATASET_DIR_TMP1,DATASET_DIR_TMP2,"MASKED")

## Generate Masked Photos

### Download required dependency and extract to folder

In [ ]:
os.chdir(DATASET_DIR_UNPREP)
tf = tarfile.open(wget.download(MASKS_DATASET))
tf.extractall()

### Cycle through photos and place mask images

In [ ]:
model = "cnn" # Select facial recognition model; Available options areh 'hog' or 'cnn'. Hog is faster, cnn is more accurate.

input_folder_list = [os.path.join(DATASET_DIR_TMP1, f) for f in os.listdir(DATASET_DIR_TMP1) if os.path.isfile(os.path.join(DATASET_DIR_TMP1, f))]
mask_folder_list = [os.path.join(DATASET_DIR_UNPREP_MASKS, f) for f in os.listdir(DATASET_DIR_UNPREP_MASKS) if os.path.isfile(os.path.join(DATASET_DIR_UNPREP_MASKS, f))]

for i in range(len(input_folder_list)):
     # get a random mask from from the mask folder
     random_mask_file = random.choice(list(mask_folder_list))
        
     if random_mask_file.endswith(('.png', '.jpg', '.jpeg')): 
      # get the next image from the image folder
      current_image_file = input_folder_list[i]
      if current_image_file.endswith(('.png', '.jpg', '.jpeg')):
        create_mask(current_image_file, random_mask_file, model)


## Save

### Save the generated images and archive them

In [ ]:
#Return a list of items that have both images and XML files
matches = compare_intersect(DATASET_DIR_TMP1,DATASET_DIR_TMP2)

#Move the images and the annotations from the temp location
move_files(DATASET_DIR_TMP1,DATASET_DIR_UNPREP_IMG, matches)
move_files(DATASET_DIR_TMP2,DATASET_DIR_UNPREP_ANNO, matches)

### Compress Dataset and Save Dataset to Google Drive

Follow this step if you wish to archive your dataset for future use

In [ ]:
!zip -r part2-datasetv1.zip {DATASET_DIR_UNPREP}
!gsutil cp part2-datasetv1.zip {DRIVE_DEV}